## Predicting turbine energy yield (TEY) using ambient variables as features.


In [1]:
!pip install tensorflow
!pip install keras
!pip install tensorflow-gpu
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dense,Dropout
from sklearn.metrics import accuracy_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import GridSearchCV,KFold
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam,RMSprop

  Using cached tensorflow-2.13.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp39-cp39-win_amd64.whl (276.5 MB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Using cached libclang-16.0.0-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached protobuf-4.23.4-cp39-cp39-win_amd64.whl (422 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached numpy-1.24.3-cp39-cp39-win_amd64.whl (14.9 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached termcolor-2.3.0-py3-no

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
ydata-profiling 4.1.2 requires numpy<1.24,>=1.16.0, but you have numpy 1.24.3 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.3 which is incompatible.
pydantic-core 2.1.2 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.3 which is incompatible.
google-api-core 1.25.1 requires google-auth<2.0dev,>=1.21.1, but you have google-auth 2.21.0 which is incompatible.



  Using cached grpcio-1.56.0-cp39-cp39-win_amd64.whl (4.2 MB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.7.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.7.1
    Uninstalling typing-extensions-4.7.1:
      Successfully uninstalled typing-extensions-4.7.1


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\HP\\AppData\\Local\\Temp\\pip-install-b_akb1tx\\tensorflow-gpu_2f1782b0730843b882ff40ea2578dc54\\setup.py'"'"'; __file__='"'"'C:\\Users\\HP\\AppData\\Local\\Temp\\pip-install-b_akb1tx\\tensorflow-gpu_2f1782b0730843b882ff40ea2578dc54\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\HP\AppData\Local\Temp\pip-wheel-v04tw3t6'
       cwd: C:\Users\HP\AppData\Local\Temp\pip-install-b_akb1tx\tensorflow-gpu_2f1782b0730843b882ff40ea2578dc54\
  Complete output (17 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Users\HP\AppDa

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
    Running setup.py install for tensorflow-gpu: started
    Running setup.py install for tensorflow-gpu: finished with status 'error'


C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'keras.wrappers'

In [ ]:
gas = pd.read_csv("P:/DS material/DS Assignments/Solved and unsolved assignments/All DS Assignments/All Datasets for assignments/gas_turbines.csv")
gas

### EDA

In [ ]:
gas.shape

In [ ]:
gas.info()

In [ ]:
gas.describe()

In [ ]:
gas.isnull()

In [ ]:
gas.isnull().sum()

In [ ]:
gas[gas.duplicated()]

In [ ]:
gas.dtypes

In [ ]:
sns.pairplot(gas)

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(gas['TEY'], axlabel = 'Turbine Energy Yeild(TEY)' ,color= 'red')
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(gas['AP'], axlabel = 'Ambient pressure (AP)' ,color= 'violet')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(x = gas['AT'],axlabel='Ambient temperature (AT)',color='green')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(x = gas['AH'],axlabel='Ambient humidity (AH)',color='blue')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(x = gas['GTEP'], axlabel='Gas turbine exhaust pressure (GTEP)', color='indigo')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(x = gas['TIT'], axlabel='Turbine inlet temperature(TIT)',color='black')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(x = gas['TAT'],axlabel='Turbine after temperature (TAT)',color='pink')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(x = gas['CDP'],axlabel='Compressor discharge pressure(CDP)',color='purple')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(x = gas['CO'],axlabel='Carbon monoxide (CO)',color='brown')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(x = gas['NOX'],axlabel='Nitrogen oxides(NOx)')
plt.show()

### Model Building

In [ ]:
x = gas.drop(labels='TEY',axis=1)
y = gas[['TEY']]

In [ ]:
x

In [ ]:
y

In [ ]:
#Covert input data in standard form
sclar = MinMaxScaler()
scale_data = sclar.fit_transform(x)
scale_data

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(scale_data,y,test_size=0.30,random_state=42)

In [ ]:
x_train.shape,y_train.shape

In [ ]:
x_test.shape,y_test.shape

### Model Training

Turning Hyperparameter: Batch Size and epochs

In [ ]:
def regression_model():
    model = Sequential()
    model.add(Dense(12, input_dim=10,kernel_initializer='uniform',activation='relu'))
    model.add(Dropout(0.2))   
    model.add(Dense(8 ,kernel_initializer='uniform',activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1,kernel_initializer = 'uniform',activation = 'sigmoid'))
    
    oppti = Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error',optimizer = oppti,metrics=['mae','mse'])
    return model

In [ ]:
model1 = KerasRegressor(build_fn=regression_model, verbose=0)
batch_size = [10,50,100]
epochs = [40,70,100]
param_grid = dict(batch_size = batch_size,epochs = epochs)
gsv = GridSearchCV(estimator=model1, param_grid=param_grid, cv = KFold(),verbose=5)
grid_res = gsv.fit(x_train,y_train)

In [ ]:
print(grid_res.best_score_,grid_res.best_params_)

Turning Hyperparameter: Learning rate and Dropout rate

In [ ]:
def regression_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(12, input_dim=10,kernel_initializer='uniform',activation='relu'))
    model.add(Dropout(0.2))   
    model.add(Dense(8 ,kernel_initializer='uniform',activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1,kernel_initializer = 'uniform',activation = 'sigmoid'))
    adam = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error',optimizer=adam, metrics=['mae','mse'])
    return model

In [ ]:
model = KerasRegressor(build_fn=regression_model,batch_size = 100,epochs = 40,verbose = 0)
learning_rate = [0.1,0.01,0.001]
dropout_rate = [0.0,0.1,0.2]
param_grid = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)
gsv = GridSearchCV(estimator= model, param_grid=param_grid, cv=KFold() ,verbose=5)
grid_res = gsv.fit(x_train,y_train)

In [ ]:
print(grid_res.best_params_,grid_res.best_score_)

Tuning of Hyperparameter :Activation Function & Kernel Initializer

In [ ]:
def regression_model(activation_function,inti):
    model = Sequential()
    model.add(Dense(12, input_dim=10,kernel_initializer='uniform',activation='relu'))
    model.add(Dropout(0.0))   
    model.add(Dense(8 ,kernel_initializer='uniform',activation='relu'))
    model.add(Dropout(0.0))
    model.add(Dense(1,kernel_initializer = 'uniform',activation = 'sigmoid'))
    adam = Adam(learning_rate=0.1)
    model.compile(loss='mean_squared_error',optimizer=adam, metrics=['mae','mse'])
    return model

In [ ]:
model = KerasRegressor(build_fn=regression_model,batch_size = 100,epochs = 40,verbose = 0)
activation_function = ['relu', 'tanh','softmax','linear']
inti = ['uniform', 'zero','normal']
param_grid = dict(activation_function = activation_function,inti = inti)
gsv_m = GridSearchCV(estimator=model,param_grid=param_grid,cv=KFold(),verbose = 5)
grid_resu = gsv_m.fit(x_train,y_train)

In [ ]:
print(grid_resu.best_score_,grid_resu.best_params_)

Tuning of Hyperparameter :Number of Neurons in hidden layer

In [ ]:
def regression_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(12,input_dim = 10,kernel_initializer='uniform',activation='relu'))
    model.add(Dropout(0.0))
    model.add(Dense(8,kernel_initializer = 'uniform',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(1,kernel_initializer='uniform',activation= 'sigmoid'))
    adam = Adam(learning_rate= 0.1)
    model.compile(loss='mean_squared_error',optimizer=adam,metrics=['mae','mse'])
    return model

In [ ]:
model = KerasRegressor(build_fn=regression_model,batch_size = 100,epochs = 40,verbose = 0)
neuron1 = [16,12,8]
neuron2 = [12,8,4]
param_grid = dict(neuron1 = neuron1,neuron2 = neuron2)
gsvp = GridSearchCV(estimator = model,param_grid = param_grid,cv=KFold(),verbose=5)
grid_result = gsvp.fit(x_train,y_train)

In [ ]:
print(grid_result.best_score_,grid_result.best_params_)

Train a model with optimum values of hyperparameter

In [ ]:
# Best parametes
# batch_size: 100 
# epochs: 40
# dropout_rate: 0.0 
# learning_rate: 0.1
# activation_function: relu 
# inti: uniform
# neuron1: 16 
# neuron2: 12 

In [ ]:
model = Sequential()
model.add(Dense(16,input_dim = 10,kernel_initializer='uniform',activation='relu'))
model.add(Dropout(0.0))
model.add(Dense(12,kernel_initializer = 'uniform',activation = 'relu'))
model.add(Dropout(0.0))
model.add(Dense(1,kernel_initializer='uniform',activation='relu'))
optimizer = RMSprop(learning_rate=0.1)
model.compile(loss='mse',optimizer = optimizer,metrics=['mae','mse'])

In [ ]:
model.fit(x_train,y_train,batch_size=100,epochs=40)
y_pred = model.predict(x_train)

In [ ]:
model.summary()

In [ ]:
mean_absolute_error(y_train,y_pred)

In [ ]:
mean_squared_error(y_train,y_pred)

In [ ]:
# testing data
test_score = model.evaluate(x_test,y_test)

In [ ]:
y_test_pred = model.predict(x_test)

In [ ]:
mean_absolute_error(y_test,y_test_pred)

In [ ]:
mean_squared_error(y_test,y_test_pred)